# Failed Terribly at this code

# About the code 

This piece of code is divided into 2 parts <br>
---> 1. Make Chunks based on the Postal Codes and Outlines: Here we are preparing a dataframe of all the file names that we read and try to divinde them into chunks based on the postal codes and the presence of outlines. <br>
---> ---> 1 a. 

In [199]:
import PyPDF2 as p2
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
import pandas as pd
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [38]:
#define the paths of the PDF scrapped 

path = 'F:/Environmental Baseline Data/Version 3/Data/PDF'
pdfs = os.listdir(path)
pdf_paths = [str(path) + '/' + str(x) for x in pdfs]
len(pdf_paths)

1329

# 1. Make Chunks based on the Postal Codes and Outlines 

This part of the code will generate a csv file with all the PDF nammes with the fillowing details: <br> - 'file_nos': just a number to indicate the file
<br> - 'file_names': the name of the file (Postal Code.pdf)
<br> - 'file_paths' : the path of the pdf file
<br> - 'int_postals' : the integer value of the postal code
<br> - 'chunk_nos': chunk_no based on the consectutive Postal Code
    

### 1 a. Postal to Int Functions 

In [39]:
def postal_to_int(Postal):
    multiplier = 1
    odd = 1
    Postal_Int_Word = 0
    Error_Message = "Invalid Postal Code"
    
    # Reversing the string 
    Postal = Postal[::-1] 
    
    # Chcek for the length of the Postal Code
    if not(len(Postal)==6):
        return(Error_Message)
    
    for letter in list(str.strip(Postal)):
        if odd%2 == 1:
            # Check if it is an interger or not   
            # If this is not an integer then exit
            if not (str.isnumeric(letter)):
                return(Error_Message)
            
            Postal_Int_Letter = multiplier * int(letter)
            Postal_Int_Word = Postal_Int_Word + Postal_Int_Letter
            
            odd = odd + 1
            multiplier = multiplier * 10
            # print(letter, Postal_Int_Letter)
            # print(Postal_Int_Word)
        else:
            # Check if it is a character or not
            # If this is not a character then exit
            if not (str.isalpha(letter)):
                return(Error_Message)
            
            # ord('A') = 65
            Postal_Int_Letter = multiplier * (ord(letter) - 65)
            Postal_Int_Word = Postal_Int_Word + Postal_Int_Letter
            
            odd = odd + 1
            multiplier = multiplier * 26
            # print(letter, Postal_Int_Letter)
            # print(Postal_Int_Word)
            
    return Postal_Int_Word

In [40]:
# never used integer to Postal but I guess it would be useful 
# Also useful in validating if Postal to Int Function works 

def int_to_postal(Postal_Int_Word):
    Postal_Int_Word_Rem = Postal_Int_Word
    Postal = ""
    odd = 0
    Error_Message = "Invalid Postal Code"
    
    # Reversing the string 
    Postal = Postal[::-1] 
 
    while odd < 6:
        if odd%2 == 0:
            Postal_Int_Letter = Postal_Int_Word_Rem % 10
            Postal = Postal + str(Postal_Int_Letter)
            Postal_Int_Word_Rem = int(Postal_Int_Word_Rem/10)
            odd = odd + 1

        else:
            Postal_Int_Letter = (Postal_Int_Word_Rem % 26) +65
            Postal = Postal + chr(Postal_Int_Letter)
            Postal_Int_Word_Rem = int(Postal_Int_Word_Rem/26)
            odd = odd + 1
            
    # Accounts for 7 digit Postal Codes
    if Postal_Int_Word_Rem > 0:
        return(Error_Message)
        
    return Postal[::-1]
  

In [128]:
file_nos = []
file_names = []
file_paths = []
int_postals = []
file_no = 1

for pdf_path in pdf_paths:
    # additional check to get rid of the string "Thumb"
    if 'Thum' in pdf_path:
        continue
    
    file_nos.append(file_no)
    file_name = str(pdf_path.split('/')[-1])
    file_names.append(file_name)
    file_paths.append(pdf_path)
    
    postal = file_name[:-4]
    int_postal = postal_to_int(postal)
    int_postals.append(int_postal)
    
len(file_nos)
len(file_names)
len(file_paths)
len(int_postals)

1328

1328

1328

1328

### 1 b. Check if Outline is present 

In [129]:
# Here we are not saving or extracting the Outlines yet
# Here we are only checking if the extraction of the outline is possible or not 
outline_present = []

for pdf_path in pdf_paths:
    if 'Thum' in pdf_path:
        continue
        
    try:
        pdfread = p2.PdfFileReader(pdf_path)
        s = pdfread.outlines
        len_s = len(s)
    except:
        outline_present.append(0) # Error in reading the outline
        
    if len_s > 0:
        outline_present.append(1)
    elif len_s == 0:
        outline_present.append(0)
    
    # Error Code for s (=pdfraed.outlines) is -999
    # the negative number allows exclusivity from the if and the elif conditions above
    len_s = -999 

In [134]:
len(outline_present)

1328

### 1 c. Check if TOC is present 

In [135]:
# Getting the path of the TOC present
path_Texts = []

for path_pdf in pdf_paths:
    if 'Thumb' in path_pdf:
        continue
    path_Text = path_pdf.replace(" .pdf", "")
    if ".pdf" in path_Text:
        path_Text = path_pdf.replace(".pdf", "")
    #print(path_Text)
    path_Text = path_Text.replace("/PDF", "/HTML Images and Tables")
    path_Text = path_Text + "/table-of-contents.txt"
    path_Texts.append(path_Text)
    #print(path_Text)
    
len(path_Texts)
path_Texts[4]

1328

'F:/Environmental Baseline Data/Version 3/Data/HTML Images and Tables/A0U3G1/table-of-contents.txt'

In [136]:
# Here we are not saving or extracting the TOCs yet
# Here we are only checking if the extraction of the TOC is possible or not 
TOC_present = []

txt_path = "F:/Environmental Baseline Data/Version 3/Data/Text/"
TOC_present = []
txt_file_needed = []
no_HTML = 0

for path_Text in path_Texts:
    try:
        file_handle = open(path_Text,"r", encoding="utf8")
        if file_handle.mode == 'r':
            contents = file_handle.read()
            contents = contents.upper()
            if 'TABLE OF CONTENTS' in contents:
                TOC_present.append('1')
            else:
                TOC_present.append('0')
                
    except:
        TOC_present.append('0')
        no_HTML = no_HTML+1
        
        
len(TOC_present)

print("HTML File does not exist for {} files".format(no_HTML))

1328

HTML File does not exist for 189 files


In [137]:
df_outlines = pd.DataFrame({'file_nos': file_nos, 
                            'file_names': file_names, 
                            'file_paths' : file_paths, 
                            'int_postals': int_postals, 
                            'outline_present': outline_present,
                            'TOC_present': TOC_present})
df_outlines.head(5)

,file_nos,file_names,file_paths,int_postals,outline_present,TOC_present
0,1,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20300,1,1
1,1,A0H8C1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20301,1,0
2,1,A0H8C2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20302,0,0
3,1,A0H8C3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20303,0,0
4,1,A0U3G1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,52841,0,1


### 1 d. Make Chunks 

Defination of a Chunk: <br>
    1 Each chunk of the pdf should have consecutive PDF files <br>
    2 Each chunk of the files can have only one outline/TOC present  <br>
    3 The first file of a chunk should be the file to have the outline/TOC present <br>
    

In [163]:
chunk_nos = [1]
chunk = 1
consecutive_postal_codes = [0]

for i in range(1, len(int_postals)):
    new_chunk_found = 0
    
    # If conditions for new chunk based on consecutine postal codes - Ref 1 
    if int_postals[i] != int_postals[i-1] + 1:
        new_chunk_found = 1
        chunk = chunk + 1
        
        
    ## If conditions for new chunk based on outline present - Ref 2 and 3    
    if outline_present[i] == 1 and outline_present[i-1] ==1 and new_chunk_found == 0: 
        new_chunk_found = 1
        chunk = chunk + 1
        
    ## If conditions for new chunk based on TOC present - Ref 2 and 3    
    if TOC_present[i] == 1 and TOC_present[i-1] ==1 and new_chunk_found == 0: 
        new_chunk_found = 1
        chunk = chunk + 1
        
    ### Chunk not found based on any of the above conditions
    chunk_nos.append(chunk)
        
    if int_postals[i] == int_postals[i-1] + 1:
        consecutive_postal_codes.append(1)
    else:
        consecutive_postal_codes.append(0)
        
len(chunk_nos)
len(consecutive_postal_codes)
chunk

1328

1328

494

In [164]:
df_outlines['consecutive_postals'] = consecutive_postal_codes
df_outlines['chunk_nos'] = chunk_nos
df_outlines.head(5)
df_outlines.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Outline_Present.csv")

,file_nos,file_names,file_paths,int_postals,outline_present,TOC_present,consecutive_postals,chunk_nos
0,1,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20300,1,1,0,1
1,1,A0H8C1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20301,1,0,1,2
2,1,A0H8C2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20302,0,0,1,2
3,1,A0H8C3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20303,0,0,1,2
4,1,A0U3G1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,52841,0,1,0,3


# 2. Extract Chapters from PDFs 

### 2 a. By Outline Method 

In [ ]:
# Functtion that runs recursively to extract the outlines of the PDF files
def nested_is_instance_check(s, level, file_name, file_names, found_file_names, chapters, levels, pages):
    level +=1
    if isinstance(s, (int, list, float, complex)):
        for x in s:
            nested_is_instance_check(x, level, file_name, file_names, found_file_names, chapters, levels, pages)
    elif level < 5:
        chapters.extend([s.title])
        levels.extend([level])
        #it was observed that for a lot of PDF files we could not find the page number 
        # Hence, -999 is the error code added for those cases 
        page_error_flag = 0
        try:
            pages.extend([(pdfread.getDestinationPageNumber(s))])
        except: 
            pages.extend([-999])
            page_error_flag = 1
        file_names.append([file_name])
        
        if page_error_flag != 1:
            found_file_names.append([file_name])
        else:
            found_file_names.append(' ')

In [234]:
def find_page_no(look_file_path, keyword):
    object = p2.PdfFileReader(look_file_path)
    
    
    # get number of pages
    NumPages = object.getNumPages() 
    
    # chapter_key_index is set as 1 because we dont want to search for"0 Before First Chapter"
    # the array chapter_key_pages has first element 0 for the same reason
    chapter_key_index = 1
    chapter_key_page_Nos = [0]
    Flag_TOC_over = 0
    
    # extract text and do the search
    for i in range(0, NumPages):
        PageObj = object.getPage(i)
        #print("this is page " + str(i)) 
        Text = PageObj.extractText()
        Text = Text.lower()
        Text = Text.replace("\n", "")
        # Text = Text
        # print (Text)
            
        #ResSearch = re.search(chapter_keys[chapter_key_index], Text)
        key = keyword.lower()
        if (key in Text):
            return i, str(look_file_path.split('/')[-1])
        
    return(-999, "")

In [252]:
# Variables for Initialization of the data frame for Index 4 - Chapters.csv
file_names = []
found_file_names = []
chapters = []
levels = []
pages = []
all_df_chapters = pd.DataFrame({'file_names': file_names, 'found_file_names': found_file_names, 'chapters': chapters, 'pages' : pages, 'levels' : levels})

for current_chunk in range(10,12):#chunk+1):
    current_df_outlines = df_outlines[df_outlines.chunk_nos == current_chunk]
    
    chunk_has_outline = 0
    chunk_noOutline_hasTOC = 0
    
    file_name_containing_index = '0'
    
    p_file_names = []
    p_found_file_names = []
    p_chapters = []
    p_levels = []
    p_pages = []
    level = 0
    # Going through all the PDFs - Chunk by Chunk
    for i in range(len(current_df_outlines)):
        # First file in the chunk has Outline 
        if current_df_outlines.iloc[i]['outline_present'] == 1:
            chunk_has_outline = 1
            print("stage 1")
            pdfread = p2.PdfFileReader(current_df_outlines.iloc[i]['file_paths'])
            s = pdfread.outlines
            
            nested_is_instance_check(s, 
                                     level, 
                                     current_df_outlines.iloc[i]['file_names'], 
                                     p_file_names, p_found_file_names, 
                                     p_chapters, p_levels, p_pages)
            file_name_containing_index = current_df_outlines.iloc[i]['file_names']
            continue
            
        if count_occorances(p_pages, -999) > 1 and chunk_has_outline == 1:
            print("stage 2: file is {}".format(current_df_outlines.iloc[i]['file_names']))
            start_flag = 0
            page_no = -999
            #print(p_pages)
            for j in range(len(p_chapters)):
                if p_pages[j] >= 0 and p_levels[j] < 5: # probably could have also used p_pages[j] >= - 998
                    print("start flag is on")
                    start_flag = 1
                    continue
                
                if start_flag != 1:
                    continue
                    
                if start_flag == 1 and p_pages[j] <= 0 and p_levels[j] < 5:
                    print("stage 3: finding page no for {} with page{}".format(p_chapters[j], p_pages[j]))
                    page_no, found_file_name = find_page_no(current_df_outlines.iloc[i]['file_paths'], 
                                                            p_chapters[j])
                    p_pages[j] = page_no
                    p_found_file_names = current_df_outlines.iloc[i]['file_names']
                    
                if page_no == -999 and p_levels[j] < 5:
                    print("Not found")
                    break # if the first keyword is not found then dont look any further
                else:
                    print("New Found ")
            
    df_chapters = pd.DataFrame({'file_names': p_file_names, 'found_file_names': p_found_file_names, 'chapters': p_chapters, 'pages' : p_pages, 'levels' : p_levels})
    all_df_chapters = pd.concat([all_df_chapters, df_chapters])
        
    
    

stage 1
stage 1
stage 1


In [238]:
len(all_df_chapters)

392

In [241]:
df_outlines['outline_present'] = outline_present

In [244]:
##########################################################################################
########## We might need to think of a better way to extract important chapters ##########
##########  df.chapters.levels < 5, is not teh best way ##################################
##########################################################################################
#all_df_chapters = all_df_chapters[all_df_chapters.chapters is not None]  #  and all_df_chapters.levels < 5]
all_df_chapters.head(10)
all_df_chapters.to_csv("F:/Environmental Baseline Data/Version 3/Indices/New try Index 4 (Outlines)- Chapters.csv")

  
 

,file_names,found_file_names,chapters,pages,levels
0,[A0H8C1.pdf],A0H8C3.pdf,Environmental Impact Assessment and Mitigation...,-999.0,2.0
1,[A0H8C1.pdf],A0H8C3.pdf,Introduction,-999.0,3.0
2,[A0H8C1.pdf],A0H8C3.pdf,Project Background,-999.0,4.0
3,[A0H8C1.pdf],A0H8C3.pdf,Regulatory Setting,-999.0,4.0
4,[A0H8C1.pdf],A0H8C3.pdf,,-1.0,5.0
5,[A0H8C1.pdf],A0H8C3.pdf,,-1.0,6.0
6,[A0H8C1.pdf],A0H8C3.pdf,,-1.0,7.0
7,[A0H8C1.pdf],A0H8C3.pdf,,-1.0,8.0
8,[A0H8C1.pdf],A0H8C3.pdf,Figure 13.1-1EnCana Ekwan Pipeline,-1.0,9.0
9,[A0H8C1.pdf],A0H8C3.pdf,Scope of Environmental Assessment,-999.0,4.0


# Table Extraction by TOC 

In [253]:
# Functtion that runs recursively to extract the outlines of the PDF files
def extract_TOC_page_numbers(path_Text):
    with open(path_Text) as json_file:
        data = json.load(json_file)
        raw_TOC_text = data['raw_text'] #currently the array has a lot of strings which are not main headers 
        if len(raw_TOC_text) < 1:
            print("No TOC Found")
    
    
    chapter_keys = ["0 Before First Chapter"] 
    Flag_Appendix_Found = 0
    for head in raw_TOC_text:
        head = head.strip()
        head = re.sub(' +', ' ',head)
        if head[0].isdigit() and len(head) > 5: # assuming all the headings in the TOC are numbered
            head = head_clean(head) # removing soo many filler '.' in the string
            
            ##########################################
            # Here, we night need to set a threshold (On the number of dots) value depending on the PDF and its TOC 
            ##########################################
            
            threshold = 2
            if head.count('.') < threshold:
                head = head.strip()
                chapter_keys.append(head)
                
            # Here we check for Appendix 
        if "Appendix" in head or "APPENDIX" in head:
            chapter_keys.append("99 Appendix")
            Flag_Appendix_Found = 1
            break
                
    if Flag_Appendix_Found == 0:
        chapter_keys.append("99 Appendix")

In [ ]:
# Variables for Initialization of the data frame for Index 4 - Chapters.csv
file_names = []
found_file_names = []
chapters = []
levels = []
pages = []
all_df_chapters = pd.DataFrame({'file_names': file_names, 'found_file_names': found_file_names, 'chapters': chapters, 'pages' : pages, 'levels' : levels})

for current_chunk in range(10,12):#chunk+1):
    current_df_outlines = df_outlines[df_outlines.chunk_nos == current_chunk]
    
    chunk_has_outline = 0
    chunk_noOutline_hasTOC = 0
    
    file_name_containing_index = '0'
    
    p_file_names = []
    p_found_file_names = []
    p_chapters = []
    p_levels = []
    p_pages = []
    level = 0
    # Going through all the PDFs - Chunk by Chunk
    for i in range(len(current_df_outlines)):
        # First file in the chunk has Outline 
        if current_df_outlines.iloc[i]['outline_present'] == 1:
            chunk_has_outline = 1
            print("stage 1")
            pdfread = p2.PdfFileReader(current_df_outlines.iloc[i]['file_paths'])
            s = pdfread.outlines
            
            nested_is_instance_check(s, 
                                     level, 
                                     current_df_outlines.iloc[i]['file_names'], 
                                     p_file_names, p_found_file_names, 
                                     p_chapters, p_levels, p_pages)
            file_name_containing_index = current_df_outlines.iloc[i]['file_names']
            continue
            
        if count_occorances(p_pages, -999) > 1 and chunk_has_outline == 1:
            print("stage 2: file is {}".format(current_df_outlines.iloc[i]['file_names']))
            start_flag = 0
            page_no = -999
            #print(p_pages)
            for j in range(len(p_chapters)):
                if p_pages[j] >= 0 and p_levels[j] < 5: # probably could have also used p_pages[j] >= - 998
                    print("start flag is on")
                    start_flag = 1
                    continue
                
                if start_flag != 1:
                    continue
                    
                if start_flag == 1 and p_pages[j] <= 0 and p_levels[j] < 5:
                    print("stage 3: finding page no for {} with page{}".format(p_chapters[j], p_pages[j]))
                    page_no, found_file_name = find_page_no(current_df_outlines.iloc[i]['file_paths'], 
                                                            p_chapters[j])
                    p_pages[j] = page_no
                    p_found_file_names = current_df_outlines.iloc[i]['file_names']
                    
                if page_no == -999 and p_levels[j] < 5:
                    print("Not found")
                    break # if the first keyword is not found then dont look any further
                else:
                    print("New Found ")
            
    df_chapters = pd.DataFrame({'file_names': p_file_names, 'found_file_names': p_found_file_names, 'chapters': p_chapters, 'pages' : p_pages, 'levels' : p_levels})
    all_df_chapters = pd.concat([all_df_chapters, df_chapters])
        
    
    

# Rough Work 

In [151]:
# I dont think I needed this function 
def count_occorances(lst, x):
    count = 0
    for ele in lst:
        if ele == x:
            count = count + 1
    return count

In [141]:
df_outlines[["file_names"]].head(5)

,file_names
0,A0H8C0.pdf
1,A0H8C1.pdf
2,A0H8C2.pdf
3,A0H8C3.pdf
4,A0U3G1.pdf


In [142]:
len(df_outlines)

1328

In [143]:
df_outlines[:1]

,file_nos,file_names,file_paths,int_postals,outline_present,TOC_present,consecutive_postals,chunk_nos
0,1,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20300,1,1,0,1


In [144]:
df_outlines[:1][["file_names"]]

,file_names
0,A0H8C0.pdf


In [145]:
df_outlines[:1][["outline_present"]]
(df_outlines[:1].values([outline_present]))

,outline_present
0,1


TypeError: 'numpy.ndarray' object is not callable

In [146]:
df_outlines.iloc[0]['outline_present']
df_outlines.iloc[0]['outline_present'] == 1

1

True

In [157]:
current_df_outlines = df_outlines[df_outlines.chunk_nos == 3]
type(current_df_outlines.file_paths)
chunk_file_paths = current_df_outlines.file_paths
chunk_size = len(chunk_file_paths)
chunk_size
len(current_df_outlines.file_paths)

pandas.core.series.Series

2

2